## Методы сбора и обработки данных из сети Интернет

### Парсинг HTML. Mongo DB

In [18]:
# 1) Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД
# 2) Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы
# 3*)Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [2]:
from pymongo import MongoClient
import pandas as pd
import json
from pprint import pprint

In [3]:
client = MongoClient('localhost', 27017)
db = client['PSC_3']
jobs_hh = db.jobs_hh
jobs_sj = db.jobs_sj

In [4]:
df_hh = pd.read_csv('job_search_hh.csv')
df_sj = pd.read_csv('job_search_sj.csv')

In [5]:
df_hh.head()

,firm_name,vacancy_name,salary_min,salary_max,city,metro_station,prereq_shrt,tasks_shrt,publish_date,vacancy_link,site
0,HeadHunter::Технический департамент,Big Data Engineer,250000.0,NaN,Москва,Алексеевская,Уверенные знания принципов работы баз данных и...,"Создание ETL-процессов, используя внутренние и...",23 сентября,https://hh.ru/vacancy/32182817?query=Data%20sc...,hh.ru
1,ООО Диджитал медикэл оперейшнс,Senior Data Scientist / NLP инженер,180000.0,215000.0,Москва,NaN,Хорошие навыки в области классического NLP (pr...,Разработка системы анализа/понимания неструкту...,24 сентября,https://hh.ru/vacancy/33458182?query=Data%20sc...,hh.ru
2,BestDoctor,Data Scientist,NaN,NaN,Москва,Парк Победы,Хорошие знания математической статистики и тео...,Модель скоринга. Прогнозирование финансовых ри...,24 сентября,https://hh.ru/vacancy/33282290?query=Data%20sc...,hh.ru
3,Сбербанк для экспертов,Руководитель направления по исследованию данны...,NaN,NaN,Москва,NaN,Data Engineers с опытом обработки потоковых да...,Моделирование: Feature Enginering: - методы оц...,24 сентября,https://hh.ru/vacancy/33747160?query=Data%20sc...,hh.ru
4,Skillbox,Data Scientist,NaN,NaN,Москва,NaN,Опыт работы с python 3.6+ (со стеком библиотек...,Проверка качества данных. Написание скриптов д...,24 сентября,https://hh.ru/vacancy/32742065?query=Data%20sc...,hh.ru


In [6]:
df_sj.head()

,firm_name,vacancy_name,salary_min,salary_max,city,metro_station,prereq_shrt,tasks_shrt,publish_date,vacancy_link,site
0,Business connection,"Программист С++, Golang, PHP, Python, DevOPS (...",100000,250000.0,Москва,NaN,Знание языка программирования C++ на продвинут...,О компании. Написание с нуля без использования...,вчера,https://www.superjob.ru/clients/business-conne...,superjob.ru
1,НТИМИ,Программист на python,100000,250000.0,Москва,NaN,"Опыт программирования на Python от 2 лет, базо...",Вам предстоит разрабатывать автоматизированную...,20 сентября,https://www.superjob.ru/vakansii/programmist-n...,superjob.ru
2,ВНИИМС,Аналитик (инженер-математик),80000,NaN,Москва,Озёрная,"Высшее образование в области математики, техни...",Разработка прогнозных моделей в части оценки п...,вчера,https://www.superjob.ru/vakansii/analitik-3256...,superjob.ru
3,Тинькофф,Специалист Отдела Операционного планирования и...,50000,NaN,Москва,NaN,"Логическое, абстрактное мышление, умение сопос...",Сбор и анализ информации по работе подразделен...,вчера,https://www.superjob.ru/clients/tinkoff-160502...,superjob.ru
4,МобильныеТелеСистемы,Старший инженер эксплуатации ТВ платформ,50000,NaN,Москва,Пражская,NaN,Образование высшее в области связи и радиотехн...,вчера,https://www.superjob.ru/clients/mts-81084.html,superjob.ru


In [17]:
# Clear all data in db
def clear_db(db):
    db.delete_many({})
    
# Add all data to db
def df_to_db(df, db_collection):
    data = json.loads(df.T.to_json()).values()
    db_collection.insert_many(data)

# Update Mongo database with upsert
def update_db(df, db):
    data = json.loads(df.T.to_json()).values()
    for item in data:
        db.update_one({'vacancy_link':item['vacancy_link']},
                      {'$set':item}, upsert = True)

# Searching for salary
def show_search_results(db, amount=200000, func='gt', params):
    n = db.find({'salary_max':{f'${func}':amount}}, params)
    for item in n:
        pprint(item)

In [24]:
# clear_db(jobs_hh)
# clear_db(jobs_sj)

# df_to_db(df_hh, jobs_hh)
# df_to_db(df_sj, jobs_sj)

# pprint(show_search_results(jobs_hh, amount, func, params))
# pprint(show_search_results(jobs_sj, amount, func, params))

update_db(df_hh, jobs_hh)
update_db(df_sj, jobs_sj)

In [ ]:
params = {'firm_name', 'vacancy_name', 'salary_min', 'salary_max', 'city',
       'metro_station', 'prereq_shrt', 'tasks_shrt', 'publish_date',
       'vacancy_link', 'site'}

In [25]:
pprint(show_search_results(jobs_hh, amount, func, params))

{'_id': ObjectId('5d8abe7f2e73b8eefa63e26e'),
 'city': 'Москва',
 'firm_name': 'ООО Диджитал медикэл оперейшнс',
 'metro_station': None,
 'prereq_shrt': 'Хорошие навыки в области классического NLP (preprocessing, '
                'NER, IR, spellchecking). Хорошие навыки в области машинного '
                'обучения. Опыт программирования на python...',
 'publish_date': '24 сентября',
 'salary_max': 215000.0,
 'salary_min': 180000.0,
 'site': 'hh.ru',
 'tasks_shrt': 'Разработка системы анализа/понимания неструктурированного '
               'текста (медицина). Прототипирование и имплементация в продакшн '
               'алгоритмов/научных статей/экспериментов.',
 'vacancy_link': 'https://hh.ru/vacancy/33458182?query=Data%20scientist',
 'vacancy_name': 'Senior Data Scientist / NLP инженер'}
{'_id': ObjectId('5d8abe7f2e73b8eefa63e281'),
 'city': 'Москва ',
 'firm_name': ' Связной',
 'metro_station': 'Автозаводская',
 'prereq_shrt': 'Хорошее владение языком R (предпочтительно) или Pyth

In [26]:
pprint(show_search_results(jobs_sj, amount, func, params))

{'_id': ObjectId('5d8abe7f2e73b8eefa63e344'),
 'city': 'Москва',
 'firm_name': 'Business connection',
 'metro_station': None,
 'prereq_shrt': 'Знание языка программирования C++ на продвинутом уровне. '
                'Хорошее знание одного из языков - Golang, PHP, Python. Знание '
                'react…',
 'publish_date': 'вчера',
 'salary_max': 250000.0,
 'salary_min': 100000,
 'site': 'superjob.ru',
 'tasks_shrt': 'О компании. Написание с нуля без использования frameworks и '
               'open source нового функционала к существующей платформе на РНР',
 'vacancy_link': 'https://www.superjob.ru/clients/business-connection-72710.html',
 'vacancy_name': 'Программист С++, Golang, PHP, Python, DevOPS (релокация на '
                 'о. Бали)'}
{'_id': ObjectId('5d8abe7f2e73b8eefa63e345'),
 'city': 'Москва',
 'firm_name': 'НТИМИ',
 'metro_station': None,
 'prereq_shrt': 'Опыт программирования на Python от 2 лет, базовый опыт '
                'фронтенд разработки – HTML, JS, CSS. Опы